# Наш любимый парсер


__Не очень приоритетно:__ 

* Попробовать дотавать ссылки на "ещё квартиры" 

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install tqdm

In [21]:
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm_notebook

Делаем ссылку для парсера. 

In [4]:
districts = [1, 325, 326, 4, 5, 6, 7, 8, 9, 10, 11, 151]

rooms = ['room1=1', 'room2=1', 'room3=1', 'room4=1', 
         'room5=1', 'room6=1', 'room7=1', 'room9=1']

objt = [1,2]

In [5]:
def create_url(params):
    main_page = 'https://www.cian.ru/cat.php?'
    params_1 = 'engine_version=2&deal_type=sale&offer_type=flat'
    params_2 = '&district={district}&{room}&region={region}&object_type={objt}'.format(**params)
    return main_page + params_1 + params_2

main_urls = [ ]

for dis in districts:
    for rm in rooms:
        for ob in objt:
            params = {
                'region': 1,
                'district': dis,
                'room': rm,
                'objt': ob}
            
            main_urls.append(create_url(params))
            
len(main_urls)

192

Качаем ссылки по каждой ссылке. 

In [9]:
def get_hrefs(url, p):
    
    MAX_RETRIES = 10
    url = url + '&p={}'.format(p)
    
    session = requests.Session()
    
    adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    
    resp = session.get(url)
    # print(resp.status_code)
    soup = BeautifulSoup(resp.content)
    
    s = soup.find_all('div', {'class': '_93444fe79c--card--_yguQ'})
    hrefs = [item.find('a', {'class': 'c6e8ba5398--header--1fV2A'}).get("href") 
                         for item in s]
    return hrefs

In [23]:
hrefs = []
i = 0

for url in tqdm_notebook(main_urls):
    for p in range(1, 55):
        i += 1
        cur_hrefs = get_hrefs(url, p)

        if p > 1:
            if cur_hrefs[-1] == hrefs[-1]:
                break

        hrefs.extend(cur_hrefs)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [24]:
main_urls.index(url)

1

In [25]:
len(hrefs)

946

In [26]:
len(set(hrefs))

869

In [18]:
url = "https://www.cian.ru/sale/flat/223309049/"
req = requests.get(url)
req

<Response [200]>

In [19]:
req.status_code

200

<span content="5 279 008 ₽" itemprop="price">5 279 008 ₽</span>

In [45]:
import re
re.sub('\D', '', price.text)

'5279008'